In [2]:
import pandas as pd
import numpy as np
import gzip
import json
import sys
import seaborn as sns
import ast
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA
from scipy.stats import zscore
from pyod.models.mad import MAD

sys.path.append("../utils/")
from myFunctions import jsonGzipToDataframe, jsonGzipToDataframe2, toDommyColumns

Guardemos el CSV comprimido dentro de un dataframe Pandas

In [3]:
df = pd.read_csv('../datasource/steam_games_chewed.csv.gz')

In [ ]:
df.info()

#### TAGS 🔎

Los NaN son de tipo float
¿Cuánto representarán del total de valores en la columna 'tags'?

In [4]:
# Crear un nuevo DataFrame solo con las filas que tienen tipo string y no NaN Float en la columna
# 'tags'
dfSinTagsNaN = df[df['tags'].apply(lambda x: isinstance(x, str))]
dfConTagsNaN = df[df['tags'].apply(lambda x: isinstance(x, float))]
print('String son', dfSinTagsNaN.shape[0])
print('NaN son', dfConTagsNaN.shape[0])

String son 31963
NaN son 162


Entonces nos quedearemos con los juegos cuyos 'tags' no sea uno de esos NaN Float

In [5]:
df = dfSinTagsNaN.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31963 entries, 0 to 31962
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     23961 non-null  object 
 1   genres        28820 non-null  object 
 2   app_name      31962 non-null  object 
 3   title         29915 non-null  object 
 4   url           31963 non-null  object 
 5   release_date  29898 non-null  object 
 6   tags          31963 non-null  object 
 7   reviews_url   31963 non-null  object 
 8   specs         31303 non-null  object 
 9   price         30618 non-null  float64
 10  early_access  31963 non-null  bool   
 11  id            31963 non-null  int64  
 12  developer     28693 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 3.0+ MB


Obtengamos las columnas dummies de la columna 'tags'

In [6]:
# Pero antes sustituyamos los NaN de la columna 'tags' por listas vacias
df['tags'] = df['tags'].fillna('[]')

# Obtengamos las columnas dummies de la columna 'tags'
dummyColumnsTags = toDommyColumns(df,'tags')

# Seamos consistentes con los nombres de columnas
dummyColumnsTags.columns = dummyColumnsTags.columns.str.lower().str.replace(' ', '_') 

print(f'Ahora tenemos {dummyColumnsTags.shape[1]} columnas dummies')

Veamos cuáles son las columnas dummies 'tags' más valiosas

In [8]:
# Sumemos los valores de la columna 'tags', los ordenamos en orden descendente, seleccionamos los 20
# valores más altos y luego los normalizamos dividiéndolos por la longitud del DataFrame original
dummyColumnsTags.sum().sort_values(ascending = False).head(20) / len(dummyColumnsTags)

tags_indie               0.551137
tags_action              0.406783
tags_adventure           0.307449
tags_casual              0.304571
tags_simulation          0.241780
tags_strategy            0.236711
tags_rpg                 0.185152
tags_singleplayer        0.135907
tags_free_to_play        0.075181
tags_multiplayer         0.074555
tags_great_soundtrack    0.069893
tags_puzzle              0.066014
tags_early_access        0.060914
tags_2d                  0.060852
tags_atmospheric         0.060476
tags_vr                  0.056190
tags_sports              0.049964
tags_platformer          0.045334
tags_story_rich          0.045177
tags_sci-fi              0.043394
dtype: float64

Nos quedaremos con las columnas cuyo peso sea >= 0,07

In [9]:
tagsColumns = ['tags_indie', 'tags_action', 'tags_adventure', 'tags_casual', 'tags_simulation']
tagsColumns += ['tags_strategy', 'tags_rpg', 'tags_singleplayer', 'tags_free_to_play', 'tags_multiplayer']
dummyTags = dummyColumnsTags[tagsColumns]

In [10]:
dummyTags.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31963 entries, 0 to 31962
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   tags_indie         31963 non-null  int64
 1   tags_action        31963 non-null  int64
 2   tags_adventure     31963 non-null  int64
 3   tags_casual        31963 non-null  int64
 4   tags_simulation    31963 non-null  int64
 5   tags_strategy      31963 non-null  int64
 6   tags_rpg           31963 non-null  int64
 7   tags_singleplayer  31963 non-null  int64
 8   tags_free_to_play  31963 non-null  int64
 9   tags_multiplayer   31963 non-null  int64
dtypes: int64(10)
memory usage: 2.7 MB


In [ ]:
dummyTags.head()

In [11]:
# Insertemos las columnas dummies al dataframe original
df = pd.concat([df, dummyTags], axis = 1)

In [ ]:
df.info()

In [ ]:
df.head(3)

#### GENRES 🔎

Los NaN son de tipo float
¿Cuánto representarán del total de valores en la columna 'genres'?

In [12]:
# Crear un nuevo DataFrame solo con las filas que tienen tipo string y no NaN Float en la columna
# 'genres'
dfSinGenresNaN = df[df['genres'].apply(lambda x: isinstance(x, str))]
dfConGenresNaN = df[df['genres'].apply(lambda x: isinstance(x, float))]
print('String son', dfSinGenresNaN.shape[0])
print('NaN son', dfConGenresNaN.shape[0])

String son 28820
NaN son 3143


Entonces nos quedearemos con los juegos cuyos 'genres' no sea uno de esos NaN Float

In [13]:
df = dfSinGenresNaN.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28820 entries, 0 to 28819
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   publisher          23910 non-null  object 
 1   genres             28820 non-null  object 
 2   app_name           28819 non-null  object 
 3   title              28819 non-null  object 
 4   url                28820 non-null  object 
 5   release_date       28802 non-null  object 
 6   tags               28820 non-null  object 
 7   reviews_url        28820 non-null  object 
 8   specs              28536 non-null  object 
 9   price              27601 non-null  float64
 10  early_access       28820 non-null  bool   
 11  id                 28820 non-null  int64  
 12  developer          28650 non-null  object 
 13  tags_indie         28820 non-null  int64  
 14  tags_action        28820 non-null  int64  
 15  tags_adventure     28820 non-null  int64  
 16  tags_casual        288

Obtengamos las columnas dummies de la columna 'genres'

In [14]:
# Obtengamos las columnas dummies de la columna 'genres'
dummyColumnsGenres = toDommyColumns(df, 'genres')

# Seamos consistentes con los nombres de columnas
dummyColumnsGenres.columns = dummyColumnsGenres.columns.str.lower().str.replace(' ', '_') 

In [ ]:
print(f'Ahora tenemos {dummyColumnsGenres.shape[1]} columnas dummies')

Veamos cuáles son las columnas dummies 'genres' más valiosas

In [15]:
# Sumemos los valores de la columna 'genres', los ordenamos en orden descendente, seleccionamos los 20
# valores más altos y luego los normalizamos dividiéndolos por la longitud del DataFrame original
dummyColumnsGenres.sum().sort_values(ascending = False).head(20) / len(dummyColumnsGenres)

genres_indie                        0.549757
genres_action                       0.392540
genres_casual                       0.286849
genres_adventure                    0.285704
genres_strategy                     0.241256
genres_simulation                   0.232200
genres_rpg                          0.190042
genres_free_to_play                 0.070472
genres_early_access                 0.050729
genres_sports                       0.043581
genres_massively_multiplayer        0.038446
genres_racing                       0.037578
genres_design_&amp;_illustration    0.015961
genres_utilities                    0.011763
genres_web_publishing               0.009299
genres_animation_&amp;_modeling     0.006350
genres_education                    0.004337
genres_video_production             0.004025
genres_software_training            0.003643
genres_audio_production             0.003227
dtype: float64

Nos quedaremos con las columnas cuyo peso sea >= 0,03

In [16]:
genresColumns = ['genres_indie', 'genres_action', 'genres_casual', 'genres_adventure']
genresColumns += ['genres_strategy', 'genres_simulation', 'genres_rpg', 'genres_free_to_play']
genresColumns += ['genres_early_access', 'genres_sports', 'genres_massively_multiplayer', 'genres_racing']
dummyGenres = dummyColumnsGenres[genresColumns]

In [ ]:
dummyGenres

In [ ]:
df.shape

In [17]:
# Insertemos las columnas dummies al dataframe original
df = pd.concat([df, dummyGenres], axis = 1)

In [ ]:
df.head(3)
#df.loc[df['title'] == 'Lost Summoner Kitty']
#df.iloc[:, 7:9]

#### YEAR 🔎

A crear la columna 'year'

In [18]:
# Extraer el año de la columna 'release_date' y crear la columna 'year'
df['year'] = df['release_date'].str.extract(r'(\d{4})')

In [19]:
# Aplicamos una mascara para marcar los valores NaN y los sumamos por columna
df['year'].isna().sum()

119

In [20]:
# Elimina las filas que tienen valor NaN en la columna 'year'
df = df.dropna(subset = ['year']).reset_index(drop = True)

In [ ]:
df.info()

In [21]:
# Convertimos el año a tipo entero
df['year'] = df['year'].astype(int)

Ok, está quedando mejor. Parece que las columnas 'url' y 'reviews_url' sobran porque no aportan nada
al análisis, vamos a quitarlas


In [22]:
# Eliminar las columnas 'url' y 'reviews_url'
df = df.drop(['url', 'reviews_url'], axis=1)

Ya conseguimos el año a partir de la columna 'release_date', vamos a borrarla. Es redundante tener a
'genres' y 'tags'<br>porque las desplegamos como dummies, tambien a borrarlas. 

In [23]:
# Eliminar las columnas 'release_date', 'genres' y 'tags'
df = df.drop(['release_date', 'genres', 'tags'], axis=1)

In [ ]:
df.head(3)

Los True y False de la columna 'early_access' mejor los cambiamos a unos y ceros respectivamente

In [24]:
# Cambiar la columna 'early_access' a tipo entero en el DataFrame 'df'
df['early_access'] = df['early_access'].astype(int)

In [ ]:
# Ver el contenido de la columna 'tags' de la primera fila
type(df.iloc[0]['early_access'])

In [ ]:
df.info()

En este punto el dataframe df esta listo para ser la fuente de datos de cuatro de las cinco funciones que responderan<br>desde la API. Siendo asi, tomaremos una instantanea del estado actual de los datos y los persistiremos en un CSV. 

In [25]:
# Guardar el DataFrame en un archivo .csv comprimido con gzip
df.to_csv('../datasource/steam_games_4Api.csv.gz', compression='gzip', index = False)

#### LA CONSULTA PARA LA API

In [ ]:
def developer(desarrollador: str):
    '''Devuelve la cantidad de items y porcentaje de contenido Free por año según empresa
       desarrolladora
    '''
    # Filtrar el DataFrame por desarrollador
    dfFiltrado = df[df['developer'] == desarrollador]

    # Contar la cantidad de items por año
    cantidadItems = dfFiltrado.groupby('year').size().reset_index(name = 'Cantidad de Items')

    # Calcular el porcentaje de juegos gratuitos por año
    dfFiltrado['free'] = (dfFiltrado['tags_free_to_play'] == 1) | (dfFiltrado['price'] == 0)
    contenidoFree = dfFiltrado.groupby('year')['free'].mean().reset_index(name = 'Contenido Free')

    # Combinar los resultados en un nuevo DataFrame
    return cantidadItems.merge(contenidoFree, on = 'year')

#### Analisis de sentimientos 🫶

Busquemos pistas sobre la presencia de outliers con un histograma

In [ ]:
# Create a histogram using Seaborn
g = sns.histplot(data = df, x = 'price')
# Add labels
g.set_xlabel('Total price per game')

En el histograma podemos ver que los datos se concentran por debajo de los 50$ aproximadamente

Busquemos pistas sobre la presencia de outliers con un diagrama de caja

In [ ]:
# Create a box plot
g = sns.boxplot(data = df, x = 'price')

# Add a title and change xlabel
g.set_title('Box Plot of Total')
g.set_xlabel('Total price per game')

En el gráfico de caja podemos ver claramente la presencia de outliers en la columna ‘price’,

Ataquemos ahora con Z-Score

In [ ]:
# Calculate z-score for each data point and compute its absolute value
z_scores = zscore(df['price'])
abs_z_scores = np.abs(z_scores)

# Select the outliers using a threshold of 3
outliers = df[abs_z_scores > 3]
outliers.head()

In [ ]:
# Obtain number of outliers
print(f'Number of outliers: {len(outliers)}')

Pero no podemos fiarnos de estos 287 porque el método z-score sólo es apropiada para distribuciones normales. Como vimos en el histograma, los datos para la variable ‘price’ están sesgados hacia la derecha por lo que debemos afinar la puntería.

Busquemos pistas sobre la presencia de outliers con MAD-Z-Score

In [ ]:
# Set threshold to 3.5
mad = MAD(threshold = 3.5)

# Convert the 'total' column into a 2D numpy array
priceReshaped = df['price'].values.reshape(-1, 1)

# Generate inline and outlier labels
labels = mad.fit(priceReshaped).labels_
labels

In [ ]:
# Obtain number of outliers
print(f'Number of outliers: {labels.sum()}')

In [ ]:
developer('Poolians.com')


In [ ]:
df.info()